# Formateo de Datos
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos.

In [1]:
import pandas as pd
import numpy as np

# Lista de Municipios.
Sonora = pd.read_excel('..\\medium_data\\book_son.xlsx',sheet_name=None)
Municipios = list(Sonora.keys())

# Datos a Ordenar.
import indicators_tables
sonora = {municipio: indicators_tables.indicators_tables(Sonora, municipio) for municipio in Municipios}

## Construccion de las Tablas

### Composición Territorial. (1)

In [2]:
mun = Municipios[0]
df = sonora[mun][1].copy()
df.set_index('Unnamed: 1',inplace=True)
df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
df.columns = df.iloc[0,:]
df = df.drop(index = ['Composición territorial','Fuente: INEGI.']).replace({'-' : 100}).astype(float)
df

Composición territorial,Absoluto,Porcentaje
Unnamed: 1,,
Localidades,25.0,100.0
Urbanas,1.0,4.0
Rurales,24.0,96.0
AGEB,14.0,100.0
Urbanas,11.0,78.6
Rurales,3.0,21.4
Manzanas,223.0,100.0
Urbanas,135.0,60.5
Rurales,88.0,39.5


In [9]:
level_zero = ['Localidades', 'AGEB', 'Manzanas']
level_one = ['Total','Urbanas','Rurales']

cols = pd.MultiIndex.from_product([level_zero, level_one],names=[2,2])

def fill_df(cols : pd.MultiIndex | list):
    abs_df = pd.DataFrame(index = Municipios, columns=cols)
    per_df = pd.DataFrame(index = Municipios, columns=cols)
    for mun in Municipios:
        data = sonora[mun][1].copy()
        data.set_index('Unnamed: 1',inplace=True)
        data = data.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
        data.columns = data.iloc[0,:]
        data = data.drop(index = ['Composición territorial','Fuente: INEGI.']).replace({'-' : 100}).astype(float)
        abs_df.loc[mun,:] = list(data['Absoluto'])
        per_df.loc[mun,:] = list(data['Porcentaje'])
    return abs_df, per_df

In [10]:
compos = pd.ExcelWriter("..\\tidy_data\\ComposicionTerritorial.xlsx")

absolute, percent = fill_df(cols)
absolute.to_excel(compos, 'Absoluto')
percent.to_excel(compos, 'Porcentaje')

compos.close()

### Ubicación Geográfica (2)

In [11]:
df = sonora[mun][2].copy()
df.set_index('Unnamed: 1',inplace=True)
df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
df.columns = df.iloc[0,:]
df = df.drop(index = ['Ubicación geográfica','Fuente: INEGI.'])
df

Ubicación geográfica,Absoluto,Porcentaje
Unnamed: 1,,
Coordenadas,Entre los paralelos 29° 41’ y 29° 53’ de latit...,-
Altitud,Entre 500 y 2 200 metros.,-
Colindancias,Colinda al norte con los municipios de San Fel...,-
Superficie (Km2),368.2,0.2
Densidad de población (habitantes/km2),7,-


In [14]:
column_two = ['Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas',
               'Colindancias']
subcolumn_two = ['Latitud', 'Latitud', 'Longitud', 'Longitud', 'Altitud', 'Altitud', '']
subsubcolumn_two = ['Min (°)', 'Max (°)', 'Min (°)', 'Max (°)', 'Min (m)', 'Max (m)', '']
cols = pd.MultiIndex.from_arrays([column_two, subcolumn_two, subsubcolumn_two])
geo_son = pd.DataFrame(columns=cols, index = Municipios)

geo_son.iloc[0, 6] = df.loc['Colindancias','Absoluto']




Coordenadas                                   \
                                Latitud         Longitud         Altitud   
                                Min (°) Max (°)  Min (°) Max (°) Min (m)   
Aconchi                             NaN     NaN      NaN     NaN     NaN   
Agua Prieta                         NaN     NaN      NaN     NaN     NaN   
Alamos                              NaN     NaN      NaN     NaN     NaN   
Altar                               NaN     NaN      NaN     NaN     NaN   
Arivechi                            NaN     NaN      NaN     NaN     NaN   
...                                 ...     ...      ...     ...     ...   
Villa Pesqueira                     NaN     NaN      NaN     NaN     NaN   
Yécora                              NaN     NaN      NaN     NaN     NaN   
Gral. Plutarco Elías Calles         NaN     NaN      NaN     NaN     NaN   
Benito Juárez                       NaN     NaN      NaN     NaN     NaN   
San Ignacio Río Muerto              NaN     NaN      NaN     NaN     NaN   

                                     \
                                      
                            Max (m)   
Aconchi                         NaN   
Agua Prieta                     NaN   
Alamos                          NaN   
Altar                           NaN   
Arivechi                        NaN   
...                             ...   
Villa Pesqueira                 NaN   
Yécora                          NaN   
Gral. Plutarco Elías Calles     NaN   
Benito Juárez                   NaN   
San Ignacio Río Muerto          NaN   

                                                                  Colindancias  
                                                                                
                                                                                
Aconchi                      Colinda al norte con los municipios de San Fel...  
Agua Prieta                                                                NaN  
Alamos                                                                     NaN  
Altar                                                                      NaN  
Arivechi                                                                   NaN  
...                                                                        ...  
Villa Pesqueira                                                            NaN  
Yécora                                                                     NaN  
Gral. Plutarco Elías Calles                                                NaN  
Benito Juárez                                                              NaN  
San Ignacio Río Muerto                                                     NaN  

[72 rows x 7 columns]